In [ ]:
#  ___________________________________________________________________________
#
#  Pyomo: Python Optimization Modeling Objects
#  Copyright (c) 2008-2025
#  National Technology and Engineering Solutions of Sandia, LLC
#  Under the terms of Contract DE-NA0003525 with National Technology and
#  Engineering Solutions of Sandia, LLC, the U.S. Government retains certain
#  rights in this software.
#  This software is distributed under the 3-clause BSD License.
#  ___________________________________________________________________________
"""
Prototype for multi-experiment optimization using the Rooney-Biegler example.
Testing trace (A-optimality) objective.
"""

import pyomo.environ as pyo
from pyomo.contrib.parmest.experiment import Experiment
from pyomo.contrib.doe import DesignOfExperiments
from pyomo.common.dependencies import pandas as pd, numpy as np


def rooney_biegler_model(data, theta=None):
    model = pyo.ConcreteModel()

    if theta is None:
        theta = {'asymptote': 15, 'rate_constant': 0.5}

    model.asymptote = pyo.Var(initialize=theta['asymptote'])
    model.rate_constant = pyo.Var(initialize=theta['rate_constant'])

    # Fix the unknown parameters
    model.asymptote.fix()
    model.rate_constant.fix()

    # Add the experiment inputs
    model.hour = pyo.Var(initialize=data["hour"].iloc[0], bounds=(0, 10))

    # Fix the experiment inputs
    model.hour.fix()

    # Add the response variable
    model.y = pyo.Var(within=pyo.PositiveReals, initialize=data["y"].iloc[0])

    def response_rule(m):
        return m.y == m.asymptote * (1 - pyo.exp(-m.rate_constant * m.hour))

    model.response_function = pyo.Constraint(rule=response_rule)

    return model


class RooneyBieglerExperiment(Experiment):

    def __init__(self, data, measure_error=None, theta=None):
        self.data = data
        self.model = None
        self.measure_error = measure_error
        self.theta = theta

    def create_model(self):
        # rooney_biegler_model expects a dataframe
        data_df = self.data.to_frame().transpose()
        self.model = rooney_biegler_model(data_df, theta=self.theta)

    def label_model(self):

        m = self.model

        # Add experiment outputs as a suffix
        m.experiment_outputs = pyo.Suffix(direction=pyo.Suffix.LOCAL)
        m.experiment_outputs.update([(m.y, self.data['y'])])

        # Add unknown parameters as a suffix
        m.unknown_parameters = pyo.Suffix(direction=pyo.Suffix.LOCAL)
        m.unknown_parameters.update(
            (k, pyo.value(k)) for k in [m.asymptote, m.rate_constant]
        )

        # Add measurement error as a suffix
        m.measurement_error = pyo.Suffix(direction=pyo.Suffix.LOCAL)
        m.measurement_error.update([(m.y, self.measure_error)])

        # Add hour as an experiment input
        m.experiment_inputs = pyo.Suffix(direction=pyo.Suffix.LOCAL)
        m.experiment_inputs.update([(m.hour, self.data['hour'])])

        # For multiple experiments, add symmetry breaking constraints
        m.sym_break_cons = pyo.Suffix(direction=pyo.Suffix.LOCAL)
        m.sym_break_cons[m.hour] = None

    def get_labeled_model(self):
        self.create_model()
        self.label_model()
        return self.model


# Data Setup
data = pd.DataFrame(
    data=[[1, 8.3], [2, 10.3], [3, 19.0], [4, 16.0], [5, 15.6], [7, 19.8]],
    columns=['hour', 'y'],
)
theta = {'asymptote': 15, 'rate_constant': 0.5}
measurement_error = 0.1

# Create 2 experiments
n_exp = 2
experiment_list = []

print(f"Creating {n_exp} experiments...")
for i in range(n_exp):
    exp_data = data.loc[i, :]
    exp = RooneyBieglerExperiment(
        data=exp_data, theta=theta, measure_error=measurement_error
    )
    experiment_list.append(exp)
    print(f"  Experiment {i+1}: hour={exp_data['hour']:.1f}")

# Create DoE object with trace (A-optimality) objective
print(f"\nRunning optimization with trace (A-optimality) objective...")
doe_obj = DesignOfExperiments(
    experiment_list=experiment_list,
    objective_option='trace',
    prior_FIM=None,
    tee=False,
    _Cholesky_option=True,
    _only_compute_fim_lower=True,
)

# Run optimization
doe_obj.optimize_experiments()

# Print results
print(f"\n{'='*60}")
print("Results")
print(f"{'='*60}")
print(f"Solver Status: {doe_obj.results['Solver Status']}")
print(f"Termination Condition: {doe_obj.results['Termination Condition']}")

# Print scenario results
scenario = doe_obj.results['Scenarios'][0]
print(f"\nFIM Metrics:")
print(f"  log10 A-opt: {scenario['log10 A-opt']:.4f}")
print(f"  log10 D-opt: {scenario['log10 D-opt']:.4f}")

print(f"\nOptimal Experiment Designs:")
for exp_idx, exp in enumerate(scenario['Experiments']):
    print(f"  Experiment {exp_idx+1}:")
    for name, value in zip(exp['Experiment Design Names'], exp['Experiment Design']):
        print(f"    {name}: {value:.4f}")
print(f"{'='*60}")

Creating 2 experiments...
  Experiment 1: hour=1.0
  Experiment 2: hour=2.0

Running optimization with trace (A-optimality) objective...

Results
Solver Status: ok
Termination Condition: optimal

FIM Metrics:
  log10 A-opt: -1.9438
  log10 D-opt: 5.8611

Optimal Experiment Designs:
  Experiment 1:
    hour: 0.9728
  Experiment 2:
    hour: 10.0000


In [ ]:
print(f"Creating {n_exp} experiments...")
for i in range(n_exp):
    exp_data = data.loc[i, :]
    exp = RooneyBieglerExperiment(
        data=exp_data, theta=theta, measure_error=measurement_error
    )
    experiment_list.append(exp)
    print(f"  Experiment {i+1}: hour={exp_data['hour']:.1f}")

# Create DoE object with trace (A-optimality) objective
print(f"\nRunning optimization with trace (A-optimality) objective...")
doe_obj = DesignOfExperiments(
    experiment_list=experiment_list,
    objective_option='trace',
    prior_FIM=None,
    tee=False,
    _Cholesky_option=True,
    _only_compute_fim_lower=True,
)

In [15]:
data = pd.Series({'hour': 2.0025, 'y': 10.0})  # y value doesn't matter
# Create experiment and DOE object
exp_obj = RooneyBieglerExperiment(
    data=data, measure_error=measurement_error, theta=theta
)
doe_obj = DesignOfExperiments(experiment_list=[exp_obj], prior_FIM=None)
prior_FIM = doe_obj.compute_FIM()

final_FIM = prior_FIM

In [16]:
2*final_FIM

array([[   80.03152186,  1396.27847147],
       [ 1396.27847147, 24360.32109167]])

In [17]:
from pyomo.contrib.doe.utils import get_FIM_metrics
get_FIM_metrics(2*final_FIM)

/Users/smondal/Documents/Github/pyomo/pyomo/contrib/doe/utils.py:169: RuntimeWarning: invalid value encountered in log10
  D_opt = np.log10(det_FIM)


{'Determinant of FIM': np.float64(-1.5930485225433367e-11),
 'Trace of cov': np.float64(4.091593995442132e-05),
 'Trace of FIM': np.float64(24440.352613528103),
 'Eigenvalues': array([    0.        , 24440.35261353]),
 'Eigenvectors': array([[-0.99836137, -0.05722381],
        [ 0.05722381, -0.99836137]]),
 'log10(D-Optimality)': np.float64(nan),
 'log10(A-Optimality)': np.float64(-4.38810746740541),
 'log10(Pseudo A-Optimality)': np.float64(4.38810746740541),
 'log10(E-Optimality)': nan,
 'log10(Modified E-Optimality)': np.float64(17.645170699697957)}

In [18]:
np.linalg.inv(2*final_FIM)

array([[-1.52916379e+15,  8.76482073e+13],
       [ 8.76482073e+13, -5.02379687e+12]])